In [1]:
import os
import sys
sys.path.append("/home/pervinco/BKAI-NeoPolyp")

import cv2
import yaml
import torch
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader

from data.BKAIDataset import BKAIDataset
from utils import decode_image, decode_mask, visualize

In [2]:
if not os.path.isdir("./batch_images"):
    os.makedirs("./batch_images")

In [3]:
with open("/home/pervinco/BKAI_MetaPolyp/config.yaml", "r") as f:
    config = yaml.safe_load(f)


## Device Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_workers = min([os.cpu_count(), config["batch_size"] if config["batch_size"] > 1 else 0, 8])

## Load Dataset
train_dataset = BKAIDataset(config=config, split=config["train"])
valid_dataset = BKAIDataset(config=config, split=config["valid"])

train_dataloader = DataLoader(dataset=train_dataset, batch_size=config["batch_size"], shuffle=True, num_workers=num_workers)
valid_dataloader = DataLoader(dataset=valid_dataset, batch_size=config["batch_size"], num_workers=num_workers)

In [4]:
for i, (images, masks) in enumerate(train_dataloader):
    if i == 5:
        break

    images = images.numpy()
    masks = masks.numpy()
    for j, (image, mask) in enumerate(zip(images, masks)):
        image = decode_image(image)
        mask = np.transpose(mask, (1, 2, 0))
        mask = np.argmax(mask, axis=-1)
        mask = decode_mask(mask)

        overlay = cv2.addWeighted(image, 0.7, mask, 0.3, 0)
        result = np.hstack((image, mask, overlay))
        result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)

        cv2.imwrite(f"./batch_images/batch{i}_no{j}.png", result)